<a href="https://colab.research.google.com/github/Sourasky-DHLAB/Whisper/blob/main/whsiper_audio_video_single_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="direction:rtl" dir="rtl"><div style="direction:rtl" dir="rtl"><p style=\"text-align: right; direction: rtl; float: right;\">

![picture](https://drive.google.com/uc?export=view&id=15xQaGVrJej6UIPjfAB-QVWXLWVChPJSt)<br>
נכתב על ידי [עודד זרחיה](mailto:odedzarchia@tauex.tau.ac.il) | [Github](https://github.com/ijontichy1)<br>
[הספרייה המרכזית ע"ש סוראסקי](https://cenlib.tau.ac.il/) | [אוניברסיטת תל אביב](https://tau.ac.il/)

# **<div style="direction:rtl" dir="rtl">1. תמלול קבצי אודיו/וידאו באמצעות Whisper </div>**
<div style="direction:rtl" dir="rtl"><p style=\"text-align: right; direction: rtl; float: right;\">

[Whisper](https://openai.com/blog/whisper/) הוא מודל לזיהוי דיבור מבית [OpenAI](https://https://openai.com/) הזמין לציבור הרחב בקוד פתוח. מודל זה אומן על יותר מ-680 אלף שעות של שיחות באנגלית ובשפות רבות אחרות – בהן גם עברית.

מחברת זו תדריך אתכם בתמלול קטעי וידאו/אודיו באמצעות Whisper. תוכלו להשתמש במחברת כפי שהיא כדי לאחסן את קבצי האודיו/וידאו ואת התמליל ב-Google Drive.

לתשומת לבכם - השימוש במחברת זו ובמודל של Whisper חופשי לחלוטין וללא שום עלות. בנוסף, אין הגבלה על אורך קטעי הוידאו/אודיו שניתן לתמלל באמצעות Whisper.

# **<div style="direction:rtl" dir="rtl">2. בדוק את סוג המעבד הגרפי (GPU) </div>**
<div style="direction:rtl" dir="rtl">

סוג [המעבד הגרפי](https://https://he.wikipedia.org/wiki/%D7%9E%D7%A2%D7%91%D7%93_%D7%92%D7%A8%D7%A4%D7%99) **(GPU - Graphics Processing Unit)** שאתם מקבלים ב-Google Colab מגדיר את המהירות שבה קטעי האודיו יתומללו. ככל שמספר [הפלופס](https://https://he.wikipedia.org/wiki/FLOPS) **(FLOPS - Floating Point Operations Per Second, פעולות נקודות צפות לשנייה)** גבוה יותר, כך התמלול מהיר יותר. יחד עם זאת, גם המעבד הגרפי החלש ביותר ב-Colab מסוגל להריץ כל מודל של Whisper. יש לוודא כי בחרתם ב-GPU כמאיץ חומרה עבור מחברת זו (Runtime → Change runtime type → Hardware accelerator).
</div>

|  GPU   |  GPU RAM   | FP32 teraFLOPS |     Availability   |
|:------:|:----------:|:--------------:|:------------------:|
|  T4    |    16 GB   |       8.1      |         Free       |
| P100   |    16 GB   |      10.6      |      Colab Pro     |
| V100   |    16 GB   |      15.7      |  Colab Pro (Rare)  |

<div style="direction:rtl" dir="rtl">

הריצו את התא הבא כדי לוודא את סוג המעבד הגרפי שהוקצה עבור מחברת זו. אפסו את ה-runtime של המחברת אם ברצונכם לקבל מעבד גרפי אחר (Runtime → Restart runtime).

In [ ]:
# check gpu type
!nvidia-smi -L
!nvidia-smi

GPU 0: Tesla T4 (UUID: GPU-b19ae099-736e-ff64-7f4b-e731e9d53185)
Sun Feb 19 15:00:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                       

# **<div style="direction:rtl" dir="rtl">3. התקנת ספריות </div>**
<div style="direction:rtl" dir="rtl">

הריצו את התא הבא כדי להוריד ולהתקין את הספריות הנחוצות לפעולת התמלול:

1.   [whisper](https://https://github.com/openai/whisper) (עבור תמלול)
2.   [moviepy](https://https://pypi.org/project/moviepy/) (עבור חילוץ אודיו)
3.   [imageio](https://https://pypi.org/project/imageio/) (נדרש עבור moviepy)
4.   [jiwer](https://https://pypi.org/project/jiwer/) (עבור בדיקת WER)

In [ ]:
!pip install git+https://github.com/openai/whisper.git 
!pip install moviepy
!pip3 install imageio==2.4.1
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-826r94bo
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-826r94bo
  Resolved https://github.com/openai/whisper.git to commit 7858aa9c08d98f75575035ecd6481f462d66ca27
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 42.3 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230124-py3-none-any.whl size=1179424 sha256=449bc97de8ede35531a21e69c105689f0291c0cd6da68de4f4d2eb7743f4714a
  Stored in directory: /tmp/pip-ephem-wheel-cache-gzexqnf_/wheels/a7/70/18/b7693c07b1d18b3dafb328f5d0496aa0d41a9c09ef332fd8e6
Successfully built opena

# **<div style="direction:rtl" dir="rtl">4. ייבוא ספריות </div>**
<div style="direction:rtl" dir="rtl">

הריצו את התא הבא כדי לייבא את הספריות הנדרשות עבור פעולת התמלול.

In [ ]:
import whisper
import moviepy.editor
from jiwer import wer

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1941504/45929032 bytes (4.2%)4317184/45929032 bytes (9.4%)6660096/45929032 bytes (14.5%)8896512/45929032 bytes (19.4%)11132928/45929032 bytes (24.2%)14327808/45929032 bytes (31.2%)16867328/45929032 bytes (36.7%)19038208/45929032 bytes (41.5%)21897216/45929032 bytes (47.7%)24502272/45929032 bytes (53.3%)27336704/45929032 bytes (59.5%)29728768/45929032 bytes (64.7%)321

# **<div style="direction:rtl" dir="rtl">5. חיבור ל-Google Drive </div>**
<div style="direction:rtl" dir="rtl">

הריצו את התא הבא כדי להתחבר ל-Google Drive האישי שלכם. כדי לראות את הקבצים שלכם פתחו את סייר הקבצים בתפריט השמאלי.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


# **<div style="direction:rtl" dir="rtl">6. פתיחת קטע הוידאו עם moviepy </div>**
<div style="direction:rtl" dir="rtl">

טענו קובץ וידאו מ-Google Drive באמצעות moviepy. שנו את הנתיב כדי לטעון את הקובץ שלכם (אתרו את הקובץ בסייר הקבצים, לחצו על הכפתור הימני בעכבר ← Copy path).

In [ ]:
video = moviepy.editor.VideoFileClip("/content/gdrive/MyDrive/tamar/GMT20200913-121425_------_avo_1280x720.mp4")

# **<div style="direction:rtl" dir="rtl">7. חילוץ שכבת אודיו </div>**
<div style="direction:rtl" dir="rtl">

הריצו את התא הבא כדי לחלץ את שכבת האודיו מקובץ הוידאו שלכם באמצעות moviepy וכדי לשמור אותה כקובץ נפרד ב-Google Drive שלכם.

In [ ]:
audio = video.audio
audio.write_audiofile("/content/gdrive/MyDrive/tamar1.mp3")

[MoviePy] Writing audio in /content/gdrive/MyDrive/tamar1.mp3


100%|██████████| 55736/55736 [01:01<00:00, 911.73it/s]


[MoviePy] Done.


# **<div style="direction:rtl" dir="rtl">8. תמלול </div>**
<div style="direction:rtl" dir="rtl">

הריצו את התא הבא כדי לתמלל את קובץ האודיו שחילצנו באמצעות Whisper. יושם לב כי אנו משתמשים במודל הגדול ביותר (large) שהוא המדויק ביותר לעברית. עבור שפות אחרות ניתן להחליף בין סוגי המודלים השונים בהתאם למפורט בטבלה הבאה:
</div>

|  Size  | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
|:------:|:----------:|:------------------:|:------------------:|:-------------:|:--------------:|
|  tiny  |    39 M    |     `tiny.en`      |       `tiny`       |     ~1 GB     |      ~32x      |
|  base  |    74 M    |     `base.en`      |       `base`       |     ~1 GB     |      ~16x      |
| small  |   244 M    |     `small.en`     |      `small`       |     ~2 GB     |      ~6x       |
| medium |   769 M    |    `medium.en`     |      `medium`      |     ~5 GB     |      ~2x       |
| large  |   2870 M   |        N/A         |      `large`       |    ~10 GB     |       1x       |


In [ ]:
model = whisper.load_model("medium")
result = model.transcribe("/content/gdrive/MyDrive/tamar1.mp3", verbose = False)

100%|█████████████████████████████████████| 1.42G/1.42G [00:30<00:00, 50.4MiB/s]


Detected language: English


100%|██████████| 24682/24682 [00:19<00:00, 1248.21frames/s]


# **<div style="direction:rtl" dir="rtl">9. שמירת התמלול </div>**
<div style="direction:rtl" dir="rtl">

הריצו את התא הבא כדי לשמור את התמלול ב-Google Drive שלכם כקובץ טקסט.

In [ ]:
with open('/content/gdrive/MyDrive/tamar/tamar1.txt', 'w') as f:
    f.write(result["text"])

# **<div style="direction:rtl" dir="rtl">10. בדיקת WER (אופציונלי) </div>**
<div style="direction:rtl" dir="rtl">

הרצת התאים הבאים תאפשר לכם להעריך את הדיוק של התמלול של Whisper במספר שלבים, וזאת במידה ויש ברשותכם את קובץ התמלול המקורי:



1.   נירמול פלט התמלול של Whisper וקובץ התמלול המקורי ([Ground Truth](https://https://en.wikipedia.org/wiki/Ground_truth)) - הסרת סימני פיסוק, רווחים מיותרים, קפיטליזציה וכו'.
2.   השוואה בין שני הקבצים וחישוב אחוז השגיאות בהתאם למדד [Word Error Rate](https://https://en.wikipedia.org/wiki/Word_error_rate) - באמצעות ספריית jiwer.

In [ ]:
# normalize result
import string
heb_transcription_norm = open('/content/gdrive/MyDrive/hebrew_large.txt', 'r').read().lower().translate(str.maketrans('', '', string.punctuation))
print(heb_transcription_norm)

In [ ]:
# normalize ground truth
heb_ground_truth_norm = open('/content/gdrive/MyDrive/gt.txt', 'r').read().lower().strip().translate(str.maketrans('', '', string.punctuation))
print(heb_ground_truth_norm)

In [ ]:
# calculate WER
ground_truth = heb_ground_truth_norm
hypothesis = heb_transcription_norm
error = wer(ground_truth, hypothesis)
print(error)